In [ ]:
import random                           # Generates random numbers
import sys, os, glob, shutil                              # Create directories, list files
from shutil import copyfile             # Copy files from Source to Destination
import cv2                              # To flip images in data augmentation
import matplotlib.pyplot as plt         # To save the images
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

# Installation and Importings

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.7MB 44kB/s 
     |████████████████████████████████| 3.8MB 53.4MB/s 
     |████████████████████████████████| 10.6MB 50.9MB/s 
     |████████████████████████████████| 14.5MB 256kB/s 
     |████████████████████████████████| 471kB 56.6MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.4.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: grpcio 1.34.0
    Uninstalling grpcio-1.34.0:
      Successfully uninstalled grpcio-1.34.0
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.efficientnet import EfficientNetB5, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(0)

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Hacker earth competitions/Identify Snake breeds/DataSet/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Hacker earth competitions/Identify Snake breeds/DataSet/test.csv')
train_path = '/content/drive/MyDrive/Hacker earth competitions/Identify Snake breeds/DataSet/train'
test_path='/content/drive/MyDrive/Hacker earth competitions/Identify Snake breeds/DataSet/test'
new_train='/content/drive/MyDrive/Hacker earth competitions/Identify Snake breeds/DataSet/New train'

# Image Preprocessing

In [ ]:
train_data.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [ ]:
train_data['imag_id']=train_data['image_id'].apply(lambda x: x+'.jpg')

In [ ]:
test_data['imag_id']=test_data['image_id'].apply(lambda x: x+'.jpg')

In [ ]:
Y=train_data['breed']

In [ ]:
X_train, X_valid= train_test_split(train_data, test_size=0.15, random_state=550)

In [ ]:
idg = ImageDataGenerator(rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,)

In [ ]:
pix1=360
pix2=360

In [ ]:
train_data_generator = idg.flow_from_dataframe(X_train,
                                               target_size=(pix1,pix2),
                                               directory = new_train,
                                               x_col = "imag_id",
                                               y_col = "breed",
                                               class_mode = "categorical",
                                               shuffle = True)

Found 4628 validated image filenames belonging to 35 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 53 invalid image filename(s) in x_col="imag_id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
valid_data_generator  = idg.flow_from_dataframe(X_valid,
                                                target_size=(pix1,pix2),
                                                directory = new_train,
                                                x_col = "imag_id",
                                                y_col = "breed",
                                                class_mode = "categorical",
                                                shuffle = True)

Found 814 validated image filenames belonging to 35 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 13 invalid image filename(s) in x_col="imag_id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
test_generator=test_datagen.flow_from_dataframe(dataframe=test_data,
                                            target_size=(pix1,pix2),
                                            directory='../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test',
                                            x_col="imag_id",
                                            y_col=None,
                                            shuffle=False,
                                            class_mode=None
                                            )

Found 0 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2361 invalid image filename(s) in x_col="imag_id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


# Metrics

In [ ]:
#acne_model.summary()
from keras import backend as K
 
def f1_weighted(true, pred): #shapes (batch, 4)
 
    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 
 
 
    ground_positives = K.sum(true, axis=0)       # = TP + FN
    pred_positives = K.sum(pred, axis=0)         # = TP + FP
    true_positives = K.sum(true * pred, axis=0)  # = TP
        #all with shape (4,)
 
    precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
    recall = (true_positives + K.epsilon()) / (ground_positives + K.epsilon()) 
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)
 
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #not sure if this last epsilon is necessary
        #matematically not, but maybe to avoid computational instability
        #still with shape (4,)
 
    weighted_f1 = f1 * ground_positives / K.sum(ground_positives)
    weighted_f1 = K.sum(weighted_f1)
    return 100*weighted_f1

# inceptionV3

In [ ]:
inception_model = InceptionV3(input_shape=[128,128,3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
for each_layer in inception_model.layers:
    each_layer.trainable = False

In [ ]:
x = Flatten()(inception_model.output)

In [ ]:
custom_final_layer = Dense(units=35, activation='softmax')(x)

acne_model = Model(inputs=inception_model.input, outputs=custom_final_layer)

In [ ]:
acne_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',f1_m])

In [ ]:
history_saved = acne_model.fit(
  train_data_generator,
  validation_data=valid_data_generator,
  epochs=10,
)

Epoch 1/10
119/119 [==============================] - ETA: 0s - loss: 7.1228 - accuracy: 0.0914 - f1_m: 7.7856 

In [ ]:
icpv3_preds=acne_model.predict_generator(test_generator,verbose=1)

In [ ]:
def sub_file_return(preds):
  sub_preds=np.argmax(preds,axis=1)
  labels = (train_data_generator.class_indices)
  labels = dict((v,k) for k,v in labels.items())
  predictions = [labels[k] for k in sub_preds]
  test_name=test_data['image_id'].values
  data_pred=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])
  return data_pred

# InceptionResNetV2

In [ ]:
inceptionresnet_model = InceptionResNetV2(input_shape=[360,360,3], weights='imagenet', include_top=False)

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
for each_layer in inceptionresnet_model.layers:
    each_layer.trainable = False

In [ ]:
incep_resnet_x = Flatten()(inceptionresnet_model.output)

In [ ]:
custom_final_layer_incepresnet = Dense(units=35, activation='softmax')(incep_resnet_x)
 
incep_resnet_model = Model(inputs=inceptionresnet_model.input, outputs=custom_final_layer_incepresnet)

In [ ]:
incep_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',f1_weighted])

In [ ]:
history_saved_incep_resnet = incep_resnet_model.fit_generator(
  train_data_generator,
  validation_data=valid_data_generator,
  epochs=8
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
145/145 [==============================] - 2942s 20s/step - loss: 22.9462 - accuracy: 0.1132 - f1_weighted: 9.9520 - val_loss: 18.1530 - val_accuracy: 0.1560 - val_f1_weighted: 14.7114
Epoch 2/8
145/145 [==============================] - 2791s 19s/step - loss: 4.1830 - accuracy: 0.6361 - f1_weighted: 62.8886 - val_loss: 19.1436 - val_accuracy: 0.1609 - val_f1_weighted: 14.8362
Epoch 3/8
